In [1]:
# 1. Импорт библиотек
import pandas as pd
import numpy as np
import pickle
import math
from collections import defaultdict
import os

In [2]:
# 2. Загрузка данных и индекса
df = pd.read_csv('../shared_data/news.csv')

In [5]:
# Выберите нужный индекс (стеммированный или лемматизированный)
INDEX_FILE = '../shared_data/search_index.pkl'  # стеммированный
#INDEX_FILE = '../shared_data/lemmatized_search_index.pkl'  # лемматизированный

In [6]:
print("Загрузка индекса...")
with open(INDEX_FILE, 'rb') as f:
    index_data = pickle.load(f)

inverted_index = index_data['inverted_index']
doc_lengths = index_data['doc_lengths']

Загрузка индекса...


In [7]:
# 3. Класс поисковой системы
class VectorSearchEngine:
    def __init__(self, inverted_index, doc_lengths, total_docs):
        self.inverted_index = inverted_index
        self.doc_lengths = doc_lengths
        self.total_docs = total_docs
        self.idf_cache = {}
        self.doc_norms = {}
        
        # Предварительно вычисляем IDF для всех терминов
        self._precompute_idf()
        # Предварительно вычисляем нормы документов
        self._precompute_doc_norms()
    
    def _precompute_idf(self):
        """Предварительное вычисление IDF для всех терминов в индексе"""
        print("Вычисление IDF...")
        for term, doc_freqs in self.inverted_index.items():
            df_t = len(doc_freqs)  # количество документов с термином
            self.idf_cache[term] = math.log(self.total_docs / (1 + df_t))
    
    def _precompute_doc_norms(self):
        """Предварительное вычисление норм документов для косинусного сходства"""
        print("Вычисление норм документов...")
        # Инициализируем нулями
        doc_vectors = defaultdict(lambda: defaultdict(float))
        
        # Собираем TF-IDF векторы
        for term, doc_freqs in self.inverted_index.items():
            idf = self.idf_cache[term]
            for doc_id, freq in doc_freqs.items():
                if doc_id in self.doc_lengths and self.doc_lengths[doc_id] > 0:
                    tf = freq / self.doc_lengths[doc_id]
                    tf_idf = tf * idf
                    doc_vectors[doc_id][term] = tf_idf
        
        # Вычисляем нормы (длины векторов)
        for doc_id, vector in doc_vectors.items():
            self.doc_norms[doc_id] = math.sqrt(sum(tf_idf ** 2 for tf_idf in vector.values()))
    
    def _compute_query_vector(self, query_terms):
        """Вычисление TF-IDF вектора для запроса"""
        query_tf = defaultdict(int)
        for term in query_terms:
            query_tf[term] += 1
        
        query_vector = {}
        query_length = len(query_terms)
        
        for term, freq in query_tf.items():
            if term in self.idf_cache:
                tf = freq / query_length
                idf = self.idf_cache[term]
                query_vector[term] = tf * idf
        
        # Вычисляем норму запроса
        query_norm = math.sqrt(sum(tf_idf ** 2 for tf_idf in query_vector.values()))
        
        return query_vector, query_norm
    
    def search(self, query, preprocess_func, top_k=10):
        """
        Поиск релевантных документов по запросу
        
        Args:
            query: поисковый запрос
            preprocess_func: функция предобработки (стемминг или лемматизация)
            top_k: количество возвращаемых результатов
        """
        # Предобработка запроса
        query_terms = preprocess_func(query)
        print(f"Обработанный запрос: {query_terms}")
        
        if not query_terms:
            return []
        
        # Вычисляем вектор запроса
        query_vector, query_norm = self._compute_query_vector(query_terms)
        
        if not query_vector:
            return []
        
        # Вычисляем релевантность для каждого документа
        scores = defaultdict(float)
        
        for term, query_tf_idf in query_vector.items():
            if term in self.inverted_index:
                for doc_id, freq in self.inverted_index[term].items():
                    if doc_id not in self.doc_norms:
                        continue
                    
                    # Вычисляем TF-IDF для термина в документе
                    if self.doc_lengths[doc_id] > 0:
                        tf = freq / self.doc_lengths[doc_id]
                        doc_tf_idf = tf * self.idf_cache[term]
                        
                        # Косинусное сходство: dot product / (norm_q * norm_d)
                        scores[doc_id] += query_tf_idf * doc_tf_idf
        
        # Нормализуем по косинусному сходству
        for doc_id in scores:
            if self.doc_norms[doc_id] > 0 and query_norm > 0:
                scores[doc_id] /= (query_norm * self.doc_norms[doc_id])
        
        # Сортируем по убыванию релевантности
        sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        
        return sorted_docs

In [8]:
# 4. Функции предобработки (должны быть такими же, как при построении индекса)
def get_preprocessing_function(index_type='stemmed'):
    """
    Возвращает функцию предобработки в зависимости от типа индекса
    """
    if index_type == 'lemmatized':
        # Импортируем и используем лемматизацию
        import pymorphy3
        from nltk.corpus import stopwords
        import re
        
        morph = pymorphy3.MorphAnalyzer()
        russian_stopwords = set(stopwords.words('russian'))
        
        def preprocess_lemmatized(text):
            if pd.isna(text):
                return []
            text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ\s]', ' ', str(text))
            text = text.lower()
            tokens = text.split()
            processed_tokens = []
            for token in tokens:
                if token not in russian_stopwords and len(token) > 2:
                    lemma = morph.parse(token)[0].normal_form
                    if len(lemma) > 2:
                        processed_tokens.append(lemma)
            return processed_tokens
        
        return preprocess_lemmatized
    
    else:
        # Используем стемминг (по умолчанию)
        from nltk.stem import SnowballStemmer
        from nltk.corpus import stopwords
        import re
        
        stemmer = SnowballStemmer("russian")
        russian_stopwords = set(stopwords.words('russian'))
        
        def preprocess_stemmed(text):
            if pd.isna(text):
                return []
            text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ\s]', ' ', str(text))
            text = text.lower()
            tokens = text.split()
            processed_tokens = []
            for token in tokens:
                if token not in russian_stopwords and len(token) > 2:
                    stem = stemmer.stem(token)
                    processed_tokens.append(stem)
            return processed_tokens
        
        return preprocess_stemmed


In [9]:
# 5. Инициализация поисковой системы
print("Инициализация поисковой системы...")
total_docs = len(df)
search_engine = VectorSearchEngine(inverted_index, doc_lengths, total_docs)

Инициализация поисковой системы...
Вычисление IDF...
Вычисление норм документов...


In [10]:
# Определяем тип предобработки на основе используемого индекса
if 'lemmatized' in INDEX_FILE:
    preprocess_func = get_preprocessing_function('lemmatized')
    print("Используется лемматизированный индекс")
else:
    preprocess_func = get_preprocessing_function('stemmed')
    print("Используется стеммированный индекс")

Используется стеммированный индекс


In [11]:
# 6. Функция для красивого вывода результатов
def display_results(search_results, df, query):
    """Красивый вывод результатов поиска"""
    print(f"\n{'='*80}")
    print(f"РЕЗУЛЬТАТЫ ПОИСКА: '{query}'")
    print(f"Найдено документов: {len(search_results)}")
    print(f"{'='*80}")
    
    for i, (doc_id, score) in enumerate(search_results, 1):
        if doc_id < len(df):
            title = df.iloc[doc_id]['title']
            source = df.iloc[doc_id]['source']
            rubric = df.iloc[doc_id]['rubric']
            text_preview = df.iloc[doc_id]['text'][:300] + "..."
            
            print(f"\n#{i} (релевантность: {score:.4f})")
            print(f"Заголовок: {title}")
            print(f"Источник: {source} | Рубрика: {rubric}")
            print(f"Текст: {text_preview}")
            print(f"{'-'*80}")

In [12]:
# 7. Интерактивный поиск
def interactive_search():
    """Интерактивный режим поиска"""
    print("\n" + "="*50)
    print("ИНТЕРАКТИВНЫЙ ПОИСК НОВОСТЕЙ")
    print("="*50)
    print("Доступные команды:")
    print("  - Введите поисковый запрос для поиска")
    print("  - 'quit' или 'exit' для выхода")
    print("  - 'top N' для изменения количества результатов (по умолчанию 10)")
    print("="*50)
    
    top_k = 10
    
    while True:
        query = input("\nВведите поисковый запрос: ").strip()
        
        if query.lower() in ['quit', 'exit', 'выход']:
            print("Завершение работы поисковой системы...")
            break
        
        elif query.lower().startswith('top '):
            try:
                new_top = int(query.split()[1])
                if new_top > 0:
                    top_k = new_top
                    print(f"Количество результатов изменено на: {top_k}")
                else:
                    print("Количество результатов должно быть положительным числом")
            except:
                print("Использование: 'top N' где N - целое число")
            continue
        
        elif query:
            print(f"\nПоиск: '{query}'...")
            results = search_engine.search(query, preprocess_func, top_k=top_k)
            display_results(results, df, query)
        
        else:
            print("Введите непустой запрос")

In [13]:
# 8. Тестовые запросы для демонстрации
def test_queries():
    """Тестовые запросы для проверки работы системы"""
    test_cases = [
        "олимпийская чемпионка Загитова",
        "экономика России",
        "Москва новости",
        "технологии искусственный интеллект",
        "спортивные соревнования"
    ]
    
    print("\n" + "="*50)
    print("ТЕСТОВЫЕ ЗАПРОСЫ")
    print("="*50)
    
    for query in test_cases:
        print(f"\nТестовый запрос: '{query}'")
        results = search_engine.search(query, preprocess_func, top_k=3)
        display_results(results, df, query)

In [14]:
test_queries()


ТЕСТОВЫЕ ЗАПРОСЫ

Тестовый запрос: 'олимпийская чемпионка Загитова'
Обработанный запрос: ['олимпийск', 'чемпионк', 'загитов']

РЕЗУЛЬТАТЫ ПОИСКА: 'олимпийская чемпионка Загитова'
Найдено документов: 3

#1 (релевантность: 0.6256)
Заголовок: Загитова согласилась вести «Ледниковый период»
Источник: lenta.ru | Рубрика: Спорт
Текст: Олимпийская чемпионка по фигурному катанию  Алина Загитова  согласилась стать ведущей шоу «Ледниковый период». Об этом сообщает Sport24. Переговоры с фигуристкой осложнялись из-за того, что Загитова изначально не была готова сосредоточиться на телевидении и временно отказаться от выступлений. После ...
--------------------------------------------------------------------------------

#2 (релевантность: 0.5786)
Заголовок: Загитова похвасталась водительскими правами через два дня после совершеннолетия
Источник: lenta.ru | Рубрика: Спорт
Текст: Олимпийская чемпионка по фигурному катанию  Алина Загитова  сообщила о получении водительского удостоверения. Об этом спор